In [1]:
execfile('test_python.py')

###########
# AlGDock #
###########
Molecular docking with adaptively scaled alchemical interaction grids

in /Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/AlGDock/BindingPMF.py
last modified Thu Dec 15 20:13:39 2016
    
using 4/4 available cores
using random number seed of -1

*** Directories ***
  start: /Users/dminh/Installers/AlGDock-0.0.1/Example
  cool: /Users/dminh/Installers/AlGDock-0.0.1/Example/cool
  dock: /Users/dminh/Installers/AlGDock-0.0.1/Example/dock

>>> Decompressing tarballs
looking for:
  complex.trans.inpcrd
  complex.pdb
  ligand.mol2
  receptor.trans.inpcrd
  ligand.trans.inpcrd
  ligand.prmtop
  ligand.db
  receptor.pdb
  complex.prmtop
  ligand.rb
  receptor.prmtop
  and frcmod files
reading prmtopcrd/receptor.tar.gz
  extracted receptor.pdb
  extracted receptor.prmtop
  extracted receptor.trans.inpcrd
reading /Users/dminh/Installers/AlGDock-0.0.1/Example/prmtopcrd/ligand.tar.gz
  extracted ligand.db
  extracted /Users/dminh/In

In [2]:
self.calc_f_RL(redo=True)
print self.f_RL['grid_MBAR'][-1]
print self.f_RL['grid_MBAR'][-1].shape


>>> Binding PMF estimation, starting at Thu, 15 Dec 2016 22:52:55 +0000


/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/scipy/optimize/minpack.py:237: RuntimeWarning: xtol=0.000000 is too small, no further improvement in the approximate
  solution is possible.
  warnings.warn(msg, RuntimeWarning)


  calculated NAMD_Gas binding PMF of -203.553409 RT with cycles 0 to 0
  calculated NAMD_Gas binding PMF of -197.537899 RT with cycles 1 to 1
  calculated NAMD_Gas binding PMF of -196.847197 RT with cycles 1 to 2
  calculated NAMD_Gas binding PMF of -196.442047 RT with cycles 1 to 3
  calculated NAMD_OBC binding PMF of -60.228561 RT with cycles 0 to 0
  calculated NAMD_OBC binding PMF of -53.673489 RT with cycles 1 to 1
  calculated NAMD_OBC binding PMF of -52.985542 RT with cycles 1 to 2
  calculated NAMD_OBC binding PMF of -52.581125 RT with cycles 1 to 3
  wrote to /Users/dminh/Installers/AlGDock-0.0.1/Example/dock/f_RL.pkl.gz

Elapsed time for binding PMF estimation: 0.784 s
[  0.           0.26448151   1.59486324   2.56882095   3.14207575
   3.70936307   4.28808098   4.88170918   5.40337676   6.13399353
   8.25307312   9.61311041  11.58678613  13.84396983  15.29206995
  18.02864486  20.05896666  23.19520034  26.417702    31.00983827
  36.45365362  42.72948924  48.94089116  55.9188

To insert a thermodynamic state into docking after a replica exchange cycle:
* self.dock_Es[state_index][cycle_index] is a dictionary. New dock_Es need to be inserted and the dictionary needs to be copied.
* self.confs['dock']['samples'][state_index][cycle_index] is a list of arrays. 
If keep_intermediate is not true, most of these will be empty except for the last state.
There is a bug: the last state has one more list element than the other states.
* self.dock_protocol needs to include the new state
* self.confs['dock']['replicas'] is a list of arrays. It needs to include one more element, duplicating the neighbor.
* self.B, self.f_RL, self.stats_RL need to be cleared.

In [3]:
import numpy as np
import copy
a = 0.5

# Defines a new thermodynamic state based on the neighboring state
neighbor_ind = [a<p['a'] for p in self.dock_protocol].index(True) - 1
lambda_n = self._lambda(a, lambda_o=self.dock_protocol[neighbor_ind])

In [4]:
# For sampling importance resampling,
# this prepares a matrix for pymbar calculations with a new thermodynamic state
(u_kln_s,N_k) = self._u_kln(self.dock_Es,self.dock_protocol)
(K,L,N) = u_kln_s.shape

u_kln_n = self._u_kln(self.dock_Es,[lambda_n])[0]
L += 1
N_k = np.append(N_k,[0])

u_kln = np.zeros([K,L,N])
u_kln[:,:-1,:] = u_kln_s
for k in range(K):
    u_kln[k,-1,:] = u_kln_n[k,0,:]

(41, 41, 46)


In [14]:
# Integrate into _run_MBAR
augmented = True
import pymbar
K = len(N_k)-1 if augmented else len(N_k)
f_k_FEPF = np.zeros(K)
f_k_BAR = np.zeros(K)
W_nl = None
for k in range(K-1):
  w_F = u_kln[k,k+1,:N_k[k]] - u_kln[k,k,:N_k[k]]
  min_w_F = min(w_F)
  w_R = u_kln[k+1,k,:N_k[k+1]] - u_kln[k+1,k+1,:N_k[k+1]]
  min_w_R = min(w_R)
  f_k_FEPF[k+1] = -np.log(np.mean(np.exp(-w_F+min_w_F))) + min_w_F
  try:
    f_k_BAR[k+1] = pymbar.BAR(w_F, w_R, \
                   relative_tolerance=1.0E-5, \
                   verbose=False, \
                   compute_uncertainty=False)
  except:
    f_k_BAR[k+1] = f_k_FEPF[k+1]
    print 'Error with BAR. Using FEP.'
f_k_FEPF = np.cumsum(f_k_FEPF)
f_k_BAR = np.cumsum(f_k_BAR)
try:
  if augmented:
    f_k_BAR = np.append(f_k_BAR,[0])
  f_k_pyMBAR = pymbar.MBAR(u_kln, N_k, \
    relative_tolerance=1.0E-5, \
    verbose = False, \
    initial_f_k = f_k_BAR, \
    maximum_iterations = 20)
  f_k_MBAR = f_k_pyMBAR.f_k
  W_nl = f_k_pymbar.getWeights()
except:
  f_k_MBAR = f_k_BAR
  print 'Error with MBAR. Using BAR.'
if np.isnan(f_k_MBAR).any():
  f_k_MBAR = f_k_BAR
  print 'Error with MBAR. Using BAR.'

In [6]:
# This provides weights, linearly indexed.
weights_n = W_nl[:,-1]

# This is for conversion to 3 indicies: state, cycle, and snapshot
cum_N_state = np.cumsum([0] + list(N_k))
cum_N_cycle = np.cumsum([0] + [self.dock_Es[0][c]['MM'].shape[0] for c in range(len(self.dock_Es[0]))])

def linear_index_to_snapshot_index(ind):
  state_index = list(ind<cum_N_state).index(True)-1
  nis_index = ind-cum_N_state[state_index]
  cycle_index = list(nis_index<cum_N_cycle).index(True)-1
  nic_index = nis_index-cum_N_cycle[cycle_index]
  return (state_index,cycle_index,nic_index)

def snapshot_index_to_linear_index(state_index,cycle_index,nic_index):
  return cum_N_state[state_index]+cum_N_cycle[cycle_index]+nic_index

# Terms to copy
if self.params['dock']['pose'] > -1:
  # Pose BPMF
  terms = ['MM','misc',\
    'k_angular_ext','k_spatial_ext','k_angular_int'] + self._scalables
else:
  # BPMF
  terms = ['MM','site','misc'] + self._scalables

In [7]:
# Selects energies and snapshots
dock_Es_s = []
confs_s = []
for c in range(len(self.dock_Es[0])):
    dock_Es_c = dict([(term,[]) for term in terms])
    confs_c = []
    for n_in_c in range(len(self.dock_Es[0][c]['MM'])):
        if (cum_N_cycle[c]==0):
            (snapshot_s,snapshot_c,snapshot_n) = linear_index_to_snapshot_index(\
                np.random.choice(range(len(weights_n)), size = 1, p = weights_n)[0])
        else:
            snapshot_c = np.inf            
            while (snapshot_c>c):
                (snapshot_s,snapshot_c,snapshot_n) = linear_index_to_snapshot_index(\
                    np.random.choice(range(len(weights_n)), size = 1, p = weights_n)[0])
        for term in terms:
            dock_Es_c[term].append(self.dock_Es[snapshot_s][snapshot_c][term][snapshot_n])
        if self.params['dock']['keep_intermediate']:
            confs_c.append(self.confs['dock']['samples'][snapshot_s][snapshot_c])
    for term in terms:
        dock_Es_c[term] = np.array(dock_Es_c[term])
    dock_Es_s.append(dock_Es_c)
    confs_s.append(confs_c)

In [8]:
# Inserts additional values
self.dock_protocol.insert(neighbor_ind+1, lambda_n)
self.dock_Es.insert(neighbor_ind+1, dock_Es_s)
self.confs['dock']['samples'].insert(neighbor_ind+1, confs_s)
self.confs['dock']['replicas'].insert(neighbor_ind+1, \
    self.confs['dock']['replicas'][neighbor_ind])

In [9]:
self.calc_f_RL(redo=True)
print self.f_RL['grid_MBAR'][-1]
print [self.f_RL['grid_MBAR'][c].shape \
       for c in range(len(self.f_RL['grid_MBAR']))]


>>> Binding PMF estimation, starting at Thu, 15 Dec 2016 22:52:56 +0000
  calculated NAMD_Gas binding PMF of -203.634671 RT with cycles 0 to 0
  calculated NAMD_Gas binding PMF of -197.619161 RT with cycles 1 to 1
  calculated NAMD_Gas binding PMF of -196.928458 RT with cycles 1 to 2
  calculated NAMD_Gas binding PMF of -196.523309 RT with cycles 1 to 3
  calculated NAMD_OBC binding PMF of -60.309823 RT with cycles 0 to 0
  calculated NAMD_OBC binding PMF of -53.754751 RT with cycles 1 to 1
  calculated NAMD_OBC binding PMF of -53.066804 RT with cycles 1 to 2
  calculated NAMD_OBC binding PMF of -52.662387 RT with cycles 1 to 3
  wrote to /Users/dminh/Installers/AlGDock-0.0.1/Example/dock/f_RL.pkl.gz

Elapsed time for binding PMF estimation: 0.815 s
[  0.           0.26448151   1.59486324   2.56882095   3.14207575
   3.70936307   4.28808098   4.88170918   5.40337676   6.13399353
   8.25307312   9.61311042  11.58678595  13.84396806  15.29206738
  18.02865696  20.05902097  23.19538625  

In [10]:
# del self